# Implementation of Probabilistic Containment for Normal Distributions

In this notebook, we'll implement the probabilistic definition of containment for band depth proposed by Yohei.

In [1]:
import statdepth
from scipy.integrate import quad
import numpy as np
import pandas as pd 
import scipy.stats as stats
from tqdm import tqdm

from numpy import exp
from scipy.special import erf
from scipy.integrate import quad
from scipy.special import binom

import bigfloat
from bigfloat import BigFloat

Now we'll define our probability, $D(f \mid f_1,...,f_n) = \binom{n}{2}^{-1}\sum_{1 \leq i < j \leq n} Pr(f_i \leq f \leq f_j)$. We'll define our probabilistic band depth function to take two $1\times n$ arrays for the mean/std, respectively. Then each $f_k = N(\mu_k, \sigma^2_k)$ for $k=1,...,n$.  

In [2]:
from statdepth.testing import generate_noisy_univariate

df = generate_noisy_univariate(n=2)

We'll use random samples from $N(0, 1)$ to generate our means and stds for now. 

In [3]:
N = 5
means, stds = np.ones(N), np.random.randint(1, 15, N)

means, stds

(array([1., 1., 1., 1., 1.]), array([14, 12,  4,  6,  2]))

We'll first write a function that calculates the depth for a single normal distribution with respect to a set of normals

In [4]:
from statdepth.depth.calculations._helper import _subsequences
from scipy.stats import norm

def f_normal(z: float, parameters):
    mu_i, sigma_i, mu_j, sigma_j, mu, sigma = parameters
    
    return exp(-(mu-z)**2 / (2*sigma))*\
            (1+erf((mu_j-z)/(np.sqrt(2)*np.sqrt(sigma_j))))*\
            (1-erf((mu_i-z)/(np.sqrt(2)*np.sqrt(sigma_i))))


def _normal_depth(means, stds, curr, f):
    n = len(means)
    cols = list(range(n))
    cols.remove(curr)
    S_nj = 0
    subseq = _subsequences(cols, 2)

    for i in tqdm(range(len(subseq))):
        sequence = subseq[i]
        i, j = sequence
        
        parameters = [
            means[i], stds[i], 
            means[j], stds[j], 
            means[curr], stds[curr]
        ]
        
        integral = quad(lambda x: f(x, parameters), -np.inf, np.inf)[0]
        S_nj += integral
        
    return S_nj / binom(n, 2)

As a sanity check, we'll numerically approximate the exact triple integral with Monte Carlo simulation to make sure our simplified integral is correct

In [5]:
from scipy.integrate import tplquad

def normcdf(x, mu, sigma):
    return norm(loc=mu, scale=sigma).cdf(x)

def _normal_containment(z, parameters: list):
    mu_i, sigma_i, mu_j, sigma_j, mu, sigma = parameters
    return (normcdf(z, mu_i, sigma_i)-normcdf(z, mu, sigma)*normcdf(z, mu_j, sigma_j))*\
            norm(loc=mu, scale=sigma).pdf(z)

And finally our function that calculates depth for all functions in the set

In [6]:
def probabilistic_normal_depth(means, stds, f):
    if len(means) != len(stds):
        raise ValueError('Error, len(means) must equal len(stds)')

    depths = []
    for k in tqdm(range(len(means))):
        mc = np.delete(means, [k])
        stdsc = np.delete(stds, [k])
        
        depths.append(    
            _normal_depth(means, stds, k, f)
        )
    
    return pd.DataFrame({
        'means': means,
        'stds': stds,
        'depths': depths
    })

We can now test and plot it to see if things look correct at first glance

In [7]:
df = probabilistic_normal_depth(means, stds, _normal_containment)



Let's take a look at the depths and then visualize them

In [8]:
import plotly.graph_objects as go

def plot_normal_data(N, df):
    d = []

    largest = df.nlargest(N, columns='depths')
    
    for i, (mu, sigma) in enumerate(zip(largest['means'], largest['stds'])):
        x = np.linspace(mu - 3*sigma, mu+3*sigma, 100)
        d.append(
            go.Scatter(
                x=x, 
                y=stats.norm.pdf(x, mu, sigma), 
                mode='lines', 
                line=dict(color='red', width=1)
            )
        )
    
    df = df.drop(largest.index)
    for i, (mu, sigma) in enumerate(zip(df['means'], df['stds'])):
        x = np.linspace(mu - 3*sigma, mu+3*sigma, 100)
        d.append(
            go.Scatter(
                x=x, 
                y=stats.norm.pdf(x, mu, sigma), 
                mode='lines', 
                line=dict(color='blue', width=1)
        ))
    
    go.Figure(
        data=d,
        layout=go.Layout(title=f'Normal Distributions ({N} deepest colored in red)', showlegend=False)
    ).show()

In [9]:
# plot_normal_data(1, df)

Let's now implement the probabilistic Poisson distribution. Suppose $f_1,...,f_n$ are independent and follow a Poisson distribution with distinct parameters. Consider $f \sim Poisson(\lambda) \neq f_i \sim Poisson(\lambda_i) \neq f_j \sim Poisson(\lambda_j)$.

In [21]:
from scipy.special import gamma, gammaincc, factorial
from numpy import exp

def gammainc(a, x):
    return gamma(a) * gammaincc(a, x)

def f_poisson(lambda_i: float, lambda_f: float, lambda_j: float, lim=1000, quiet=False) -> float:
    '''
    Parameters:
    
    lambda_i: 
        mean of f_i (lower function)
    lambda_f: 
        mean of f (function to calculate probabilistic containment)
    lambda_j: 
        mean of f_j (upper function)
    lim=10000: Upper bound of discrete infinite sum
    
    Returns:
    
    float: Probability of containment
    '''
    
    s = 0
    
    for z in range(1, lim):
        num = BigFloat(np.power(lambda_f, z)*(gamma(z)-gammainc(z, lambda_j))*gammainc(1+z, lambda_i))
        denom = BigFloat(factorial(z)*gamma(z)*gamma(1+z))
        print(f'num is {num} and denom is {denom}')
        if num == 0 or denom == 0 and not quiet:
            break
        
        d = BigFloat(num / denom)
        
        s += d
        
    return exp(-lambda_f) * s


Now that we have our definition of containment, let's write the depth calculation function for a given set of data

In [22]:
def _poisson_depth(means: list, curr: int, to_compute=None):
    n = len(means)
    cols = list(range(n))
    cols.remove(curr)
    S_nj = 0
    subseq = _subsequences(cols, 2)

    for i in tqdm(range(len(subseq))):
        sequence = subseq[i]
        i, j = sequence
        lambda_i, lambda_f, lambda_j = means[i], means[curr], means[j]
        
        S_nj += f_poisson(lambda_i, lambda_f, lambda_j)
        
    return S_nj / binom(n, 2)

def probabilistic_poisson_depth(means: list, to_compute=None):
    depths = []
    for i in tqdm(range(len(means))):
        depths.append(_poisson_depth(means, i, to_compute=to_compute))
    
    return pd.DataFrame({'means': means, 'depths': depths})


Let's now plot the Poisson distributions to get a sense of which function is deepest

In [23]:
def nlargest(df, n):
    df['sums'] = df.sum(axis=1)
    return df.nlargest(n=n, columns='sums').drop('sums', axis=1).T

df = nlargest(pd.read_csv('../../experiment-testing/data/Homo_sapiens.csv').set_index('target_id'), 25)

depths = probabilistic_poisson_depth(df.mean().values)
depths

100%|██████████| 276/276 [00:00<00:00, 4859.33it/s]

100%|██████████| 276/276 [00:00<00:00, 6126.93it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and


100%|██████████| 276/276 [00:00<00:00, 5301.97it/s]

  0%|          | 0/276 [00:00<?, ?it/s]


num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 an

100%|██████████| 276/276 [00:00<00:00, 2678.39it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

...

100%|██████████| 276/276 [00:00<00:00, 3133.26it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and

100%|██████████| 25/25 [00:01<00:00, 12.79it/s]


num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 and denom is 1.0
num is 0 an

,means,depths
0,547808.166667,0.0
1,526020.333333,0.0
2,219317.666667,0.0
3,150122.900000,0.0
4,110637.350000,0.0
5,105101.666667,0.0
6,76712.200000,0.0
7,50978.416667,0.0
8,43477.233333,0.0
9,40812.583333,0.0


Since the factorial values quickly approach the maximum value for floats in Python, we'll work on a log scale instead

In [14]:
from numpy import log 
from scipy.special import loggamma


And now a version for $f_1, ... ,f_n \sim Binomial(n_i, p_i)$ is some vector of parameters for the $i$th observation. 

In [15]:
from scipy.special import hyp2f1, binom

def f_binom(params: list, lim=1000) -> float:
    n_i, p_i, n, p, n_j, p_j = params
    
    s = 0
    for z in range(1, lim):
        print(f'z is {z}')
        t = (1-p)**(n-z)*p**z*(1-p_i)**n_i*(1-p_j)**(n_j-z)*p_j**z*\
             binom(n, z)*binom(n_j, z)*\
             ((1/(1-p_i))**n_i - (1-p_i)**(-1-z)*p_i**(1+z)*binom(n_i, 1+z)*\
             hyp2f1(1,1-n_i+z, 2+z, p_i/(p_i-1)))*hyp2f1(1, -n_j+z, 1+z, p_j/(p_j-1))
        
        if np.isnan(t):
            print(t)
            break
            
    return s

In [16]:
f_binom([50,.1,3,.5,4,.3])

z is 1
z is 2
z is 3
z is 4
z is 5
z is 6
z is 7
z is 8
z is 9
z is 10
z is 11
z is 12
z is 13
z is 14
z is 15
z is 16
z is 17
z is 18
z is 19
z is 20
z is 21
z is 22
z is 23
z is 24
z is 25
z is 26
z is 27
z is 28
z is 29
z is 30
z is 31
z is 32
z is 33
z is 34
z is 35
z is 36
z is 37
z is 38
z is 39
z is 40
z is 41
z is 42
z is 43
z is 44
z is 45
z is 46
z is 47
z is 48
z is 49
z is 50
z is 51
z is 52
z is 53
z is 54
z is 55
z is 56
z is 57
z is 58
z is 59
z is 60
z is 61
z is 62
z is 63
z is 64
z is 65
z is 66
z is 67
z is 68
z is 69
z is 70
z is 71
z is 72
z is 73
z is 74
z is 75
z is 76
z is 77
z is 78
z is 79
z is 80
z is 81
z is 82
z is 83
z is 84
z is 85
z is 86
z is 87
z is 88
z is 89
z is 90
z is 91
z is 92
z is 93
z is 94
z is 95
z is 96
z is 97
z is 98
z is 99
z is 100
z is 101
z is 102
z is 103
z is 104
z is 105
z is 106
z is 107
z is 108
z is 109
z is 110
z is 111
z is 112
z is 113
z is 114
z is 115
z is 116
z is 117
z is 118
z is 119
z is 120
z is 121
z is 122
z is 123
z

0